In [1]:
# # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame

In [28]:
cities_csv = "Resources/cities.csv"

'Resources/cities.csv'

In [32]:
cities_df = pd.read_csv(cities_csv)
cities_df.dropna(inplace=True)
cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kaitangata,-46.2817,169.8464,52.00,71,95,21.00,NZ,1611806035
1,Jamestown,42.0970,-79.2353,21.99,86,90,7.54,US,1611805907
2,Gumdag,39.2061,54.5906,37.26,75,19,23.80,TM,1611806035
3,El Salvador,13.8333,-88.9167,75.20,83,0,1.14,SV,1611806035
4,Mount Gambier,-37.8333,140.7667,82.40,39,100,8.05,AU,1611806010
...,...,...,...,...,...,...,...,...,...
559,Olinda,-8.0089,-34.8553,78.80,73,75,6.91,BR,1611806178
560,Bam,29.1060,58.3570,47.17,38,0,3.22,IR,1611806178
561,Pokhara,28.2333,83.9833,64.22,26,0,4.41,NP,1611806179
562,Stokmarknes,68.5646,14.9108,24.01,63,27,5.99,NO,1611806179


### Humidity Heatmap 

In [33]:
#Configure gmaps.
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(float)

#plot heatmap
fig = gmaps.figure()

#Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10, 
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [ ]:
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

### Hotel Map

In [ ]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [ ]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
 # Add marker layer ontop of heat map


# Display figure